In [ ]:
# 06/15/2021
from __future__ import absolute_import
from IPython.display import Image
import glob, os, shutil, sys, tarfile
from os.path import *

contentPth = os.getcwd()
tensorflowImagesPth = join(contentPth, 'TensorflowImages')

try:
    from google.colab import drive, files
    if exists('/content/sample_data'):
        shutil.rmtree('/content/sample_data')
    
    drive.mount('/content/gdrive', force_remount=True)
    myDrivePth = '/content/gdrive/My Drive'
    imagesPth = join('/content/gdrive/My Drive/TensorflowImages')
    
    os.chdir(myDrivePth)
    from BashColors import C
    from TarfileFunctions import *
    os.chdir(contentPth)
    
    if not exists(tensorflowImagesPth):
        shutil.copytree(imagesPth, tensorflowImagesPth)
    else: print(f'{C.Green}{tensorflowImagesPth}{C.ColorOff} exists')
        
except ModuleNotFoundError as err: 
    # print(err)
    from BashColors import C
    from TarfileFunctions import *
    if not exists(tensorflowImagesPth):
        tar = tarfile.open(
            'New_Jovyan_Defcon_Augmented_Images.tar.gz')
        tar.extractall()
        tar.close()
    else: print(f'{C.Green}{tensorflowImagesPth}{C.ColorOff} exists')
    
try:
    import tensorflow as tf
    print(f'tensorflow: {C.IBlue}{tf.__version__}{C.ColorOff}')
except ModuleNotFoundError:
    %pip install tensorflow
    import tensorflow as tf
    
try:
    import tensorflow_datasets as tfds
except ModuleNotFoundError:
    %pip install tensorflow-datasets
    import tensorflow_datasets as tfds
    
try:
    import numpy as np
    np.set_printoptions(precision=4)
except ModuleNotFoundError:
    %pip install numpy
    import numpy as np
    np.set_printoptions(precision=4)

try:
    import PIL
    import PIL.Image
except ModuleNotFoundError:
    %pip install pillow
    import PIL
    import PIL.Image

try:
    import matplotlib as mpl
    import matplotlib.pyplot as plt
    mpl.rcParams['figure.figsize'] = (11, 11)
    mpl.rcParams['axes.grid'] = False
except ModuleNotFoundError:
    %pip install matplotlib
    import matplotlib as mpl
    import matplotlib.pyplot as plt
    mpl.rcParams['figure.figsize'] = (11, 11)
    mpl.rcParams['axes.grid'] = False

try: import graphviz
except Exception as err:
    print(err)

print(f'cwd: {C.IBlue}{os.getcwd()}')

In [ ]:
import tensorflow
physicalDevices = tensorflow.config.experimental.list_physical_devices()
tensorflow.config.experimental.set_memory_growth(physicalDevices[1], True)


In [ ]:
import tensorflow
from tensorflow.keras.utils import plot_model
mobile = tensorflow.keras.applications.mobilenet.MobileNet()
fileName = mobile.name + '.png'
print(fileName)
# mobile.summary()
import graphviz
plot_model(model=mobile,
           to_file=fileName,
           show_shapes=True,
           show_dtype=True)

print()

In [ ]:
from tensorflow.keras.preprocessing import image

def prepare_image(filePath):
    img = image.load_img(path=filePath,
                         target_size=(224, 224))
    image_array = image.img_to_array(img)
    image_array_expanded_dims = np.expand_dims(image_array, axis=0)
    return tf.keras.applications.mobilenet.preprocess_input(
        image_array_expanded_dims
    )
print('ok')

In [ ]:
from IPython.display import Image
def showImage(fromPath):
    show=Image(filename=fromPath,
               width=64, height=64,
               retina=True)
    display(show)
print('ok')

In [ ]:
from tensorflow.keras.applications import imagenet_utils
os.chdir(tensorflowImagesPth)
imageGlob=glob.glob('**', recursive=True)
for fil in sorted(imageGlob):
    if isfile(fil) and fil.endswith('.png'):
        newFilPath=abspath(fil)
        prepared_image = prepare_image(filePath=newFilPath)
        print(basename(newFilPath))
        showImage(newFilPath)
        # print(prepared_image)
        thisPrediction=mobile.predict(prepared_image)
        result = imagenet_utils.decode_predictions(thisPrediction)
        print(result)
        print()

os.chdir(contentPth)

In [ ]:
# from tensorflow.python.framework.ops import disable_eager_execution
# disable_eager_execution
# tf.config.run_functions_eagerly('True')
print('executing_eagerly:', tf.executing_eagerly())

In [ ]:
batch_size = 8
img_height = 224
img_width = 224

In [ ]:
train_ds = tf.keras.preprocessing.image_dataset_from_directory(
  tensorflowImagesPth,
  validation_split=0.2,
  subset="training",
  seed=123,
  image_size=(img_height, img_width),
  batch_size=batch_size)

In [ ]:
val_ds = tf.keras.preprocessing.image_dataset_from_directory(
  tensorflowImagesPth,
  validation_split=0.2,
  subset="validation",
  seed=123,
  image_size=(img_height, img_width),
  batch_size=batch_size)

In [ ]:
class_names=train_ds.class_names
class_names

### Visualize the data

Here are the first 9 images from the training dataset.

In [ ]:
for images, labels in train_ds.take(1):
  for i in range(8):
    ax = plt.subplot(3, 3, i + 1)
    plt.imshow(images[i].numpy().astype("uint8"))
    # title=labels[i].numpy().astype('uint8')
    title=labels[i]
    # plt.title(label=title)
    # plt.title([labels[i]])
    plt.axis("off")


You can train a model using these datasets by passing them to `model.fit` (shown later in this tutorial). If you like, you can also manually iterate over the dataset and retrieve batches of images:

In [ ]:
for image_batch, labels_batch in train_ds:
    print(image_batch.shape)
    print(labels_batch.shape)
    break

The `image_batch` is a tensor of the shape `(32, 180, 180, 3)`. This is a batch of 32 images of shape `180x180x3` (the last dimension referes to color channels RGB). The `label_batch` is a tensor of the shape `(32,)`, these are corresponding labels to the 32 images. 


Note: you can call `.numpy()` on either of these tensors to convert them to a `numpy.ndarray`.

### Standardize the data


The RGB channel values are in the `[0, 255]` range. This is not ideal for a neural network; in general you should seek to make your input values small. Here, we will standardize values to be in the `[0, 1]` by using a Rescaling layer.

In [ ]:
from tensorflow.keras import layers
from tensorflow.keras.layers.experimental.preprocessing import Rescaling
normalization_layer = Rescaling(scale=1./255)

There are two ways to use this layer. You can apply it to the dataset by calling map:

In [ ]:
normalized_ds = train_ds.map(lambda x, y: (normalization_layer(x), y))
image_batch, labels_batch = next(iter(normalized_ds))
first_image = image_batch[0]
# Notice the pixels values are now in `[0,1]`.
print(np.min(first_image), 'to', np.max(first_image)) 

Or, you can include the layer inside your model definition to simplify deployment. We will use the second approach here.

Note: If you would like to scale pixel values to `[-1,1]` you can instead write `Rescaling(1./127.5, offset=-1)`

Note: we previously resized images using the `image_size` argument of `image_dataset_from_directory`. If you want to include the resizing logic in your model, you can use the [Resizing](https://www.tensorflow.org/api_docs/python/tf/keras/layers/experimental/preprocessing/Resizing) layer instead.


### Configure the dataset for performance

Let's make sure to use buffered prefetching so we can yield data from disk without having I/O become blocking. These are two important methods you should use when loading data.

`.cache()` keeps the images in memory after they're loaded off disk during the first epoch. This will ensure the dataset does not become a bottleneck while training your model. If your dataset is too large to fit into memory, you can also use this method to create a performant on-disk cache.

`.prefetch()` overlaps data preprocessing and model execution while training. 

Interested readers can learn more about both methods, as well as how to cache data to disk in the [data performance guide](https://www.tensorflow.org/guide/data_performance#prefetching).

In [ ]:
AUTOTUNE = tf.data.AUTOTUNE

train_ds = train_ds.cache().prefetch(buffer_size=AUTOTUNE)
val_ds = val_ds.cache().prefetch(buffer_size=AUTOTUNE)

### Train a model

For completeness, we will show how to train a simple model using the datasets we just prepared. This model has not been tuned in any way - the goal is to show you the mechanics using the datasets you just created. To learn more about image classification, visit this [tutorial](https://www.tensorflow.org/tutorials/images/classification).

In [ ]:
num_classes = 2

model = tf.keras.Sequential([
  layers.experimental.preprocessing.Rescaling(1./255),
  layers.Conv2D(32, 3, activation='relu'),
  layers.MaxPooling2D(),
  layers.Conv2D(32, 3, activation='relu'),
  layers.MaxPooling2D(),
  layers.Conv2D(32, 3, activation='relu'),
  layers.MaxPooling2D(),
  layers.Flatten(),
  layers.Dense(128, activation='relu'),
  layers.Dense(num_classes)
])

In [ ]:
model.build(input_shape=(8, 224, 224, 3))
model.summary()

In [ ]:
model.compile(
  optimizer='adam',
  loss=tf.losses.SparseCategoricalCrossentropy(from_logits=True),
  metrics=['accuracy'])
# model.summary()

In [ ]:
model.fit(
  train_ds,
  validation_data=val_ds,
  epochs=2
)

In [ ]:
try: import graphviz
except ModuleNotFoundError:
    %pip install graphviz
    
tf.keras.utils.plot_model(model=model,
                          show_shapes=True,
                          expand_nested=True,
                          rankdir=contentPth)

Note: we will only train for a few epochs so this tutorial runs quickly. 

In [ ]:
mobilenet_v2 = tf.keras.applications.MobileNetV2(
    input_shape=None, alpha=1.0, include_top=False,
    weights=None, input_tensor=None, pooling=None,
    classes=3, classifier_activation='relu'
)

mobilenet_v2.compile(optimizer='Adam')
# mobilenet_v2.summary()

modelName = mobilenet_v2.name + '_no_weights_no_top.h5'
print('modelName: ', mobilenet_v2.name)
modelSavePth = join(contentPth, modelName)

if exists(contentPth):
    mobilenet_v2.save(modelSavePth)
    print(f'saved: {C.Green}{modelSavePth}{C.ColorOff}')
    
mobilenet = tf.keras.models.load_model(filepath=modelSavePth)

Note: you can also write a custom training loop instead of using `model.fit`. To learn more, visit this [tutorial](https://www.tensorflow.org/guide/keras/writing_a_training_loop_from_scratch).

You may notice the validation accuracy is low to the compared to the training accuracy, indicating our model is overfitting. You can learn more about overfitting and how to reduce it in this [tutorial](https://www.tensorflow.org/tutorials/keras/overfit_and_underfit).

## Using tf.data for finer control

The above keras.preprocessing utilities are a convenient way to create a `tf.data.Dataset` from a directory of images. For finer grain control, you can write your own input pipeline using `tf.data`. This section shows how to do just that, beginning with the file paths from the zip we downloaded earlier.

In [ ]:
data_dir=tensorflowImagesPth
searchString=join(tensorflowImagesPth, '*/*')
searchString=str(searchString)
datasetList=tf.data.Dataset.list_files(searchString)
print(f'datasetList: {len(datasetList)}')

image_count = len(datasetList)
datasetList = datasetList.shuffle(buffer_size=image_count,
                                  reshuffle_each_iteration=False,)

for fil in datasetList.take(1):
    print(fil.numpy())

In [ ]:
val_size = int(image_count * 0.2)
train_ds = datasetList.skip(val_size)
val_ds = datasetList.take(val_size)

You can see the length of each dataset as follows:

In [ ]:
print(tf.data.experimental.cardinality(train_ds).numpy())
print(tf.data.experimental.cardinality(val_ds).numpy())

Write a short function that converts a file path to an `(img, label)` pair:

In [ ]:
def get_label(file_path):
  # convert the path to a list of path components
  parts = tf.strings.split(file_path, os.path.sep)
  # The second to last is the class-directory
  one_hot = parts[-2] == class_names
  # Integer encode the label
  return tf.argmax(one_hot)

In [ ]:
def decode_img(img):
  # convert the compressed string to a 3D uint8 tensor
  img = tf.image.decode_jpeg(img, channels=3)
  # resize the image to the desired size
  return tf.image.resize(img, [img_height, img_width])

In [ ]:
def process_path(file_path):
  label = get_label(file_path)
  # load the raw data from the file as a string
  img = tf.io.read_file(file_path)
  img = decode_img(img)
  return img, label

Use `Dataset.map` to create a dataset of `image, label` pairs:

In [ ]:
# Set `num_parallel_calls` so multiple images are loaded/processed in parallel.
train_ds = train_ds.map(process_path, num_parallel_calls=AUTOTUNE)
val_ds = val_ds.map(process_path, num_parallel_calls=AUTOTUNE)

In [ ]:
for image, label in train_ds.take(1):
  print("Image shape: ", image.numpy().shape)
  print("Label: ", label.numpy())

### Configure dataset for performance

To train a model with this dataset you will want the data:

* To be well shuffled.
* To be batched.
* Batches to be available as soon as possible.

These features can be added using the `tf.data` API. For more details, see the [Input Pipeline Performance](../../guide/performance/datasets) guide.

In [ ]:
def configure_for_performance(ds):
    ds = ds.cache()
    ds = ds.shuffle(buffer_size=1000)
    ds = ds.batch(batch_size)
    ds = ds.prefetch(buffer_size=AUTOTUNE)
    return ds

train_ds = configure_for_performance(train_ds)
val_ds = configure_for_performance(val_ds)

In [ ]:
model.fit(train_ds,
          validation_data=val_ds,
          epochs=3)

In [ ]:
# get_label_name = metadata.features['label'].int2str

image, label = next(iter(train_ds))
# _ = plt.imshow(image)
_ = plt.title(get_label_name(label))

In [ ]:
train_ds = configure_for_performance(train_ds)
val_ds = configure_for_performance(val_ds)
# test_ds = configure_for_performance(test_ds)

You can find a complete example of working with the flowers dataset and TensorFlow Datasets by visiting the [Data augmentation](https://www.tensorflow.org/tutorials/images/data_augmentation) tutorial.

## Next steps

This tutorial showed two ways of loading images off disk. First, you learned how to load and preprocess an image dataset using Keras preprocessing layers and utilities. Next, you learned how to write an input pipeline from scratch using tf.data. Finally, you learned how to download a dataset from TensorFlow Datasets. As a next step, you can learn how to add data augmentation by visiting this [tutorial](https://www.tensorflow.org/tutorials/images/data_augmentation). To learn more about tf.data, you can visit this [guide](https://www.tensorflow.org/guide/data).